In [1]:

from plants_sm.data_standardization.truncation import Truncator
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.featurization.proteins.bio_embeddings.prot_bert import ProtBert

/home/jcapela/miniconda3/envs/deeplants/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import torch
from plants_sm.models.fc.fc import DNN
from plants_sm.models.pytorch_model import PyTorchModel
from torch import nn


transformers = [ProteinStandardizer(), Truncator(max_length=884), ProtBert(device="cpu")]
model = DNN(1024, [2560], 5743, batch_norm=True, last_sigmoid=True)
model.load_state_dict(torch.load("prot_bert.pt"))
model = PyTorchModel(model=model, loss_function=nn.BCELoss, model_name="ec_number")

In [16]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset


dataset = SingleInputDataset.from_csv("data/aminotransferase_binary.csv", representation_field="SEQ", instances_ids_field="ids", nrows=10)

for i, transformer in enumerate(transformers):
    transformers[i].fit(dataset)

In [17]:
transformers[i].fitted

True

In [18]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset
from plants_sm.pipeline.pipeline import Pipeline


pipeline = Pipeline(steps=transformers, models=[model])

dataset = SingleInputDataset.from_csv("data/aminotransferase_binary.csv", representation_field="SEQ", instances_ids_field="ids", nrows=10)

In [19]:
pipeline.predict(dataset)

ProtBert: 100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [21]:
pipeline.save("DNN_ProtBERT_all_data")